# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import pingouin as pg

# Import Data

Load data and remove unnecessary column(s)

In [2]:
df = pd.read_excel("Data hasil kuesioner.xlsx")
df.drop("Unnamed: 0",axis=1,inplace=True)
df.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,3,3,3,4,4,4,3,3,3,4,4,2,3,3,3,3,3,3,3,3
1,4,4,3,3,4,3,3,3,3,4,4,3,4,3,4,3,3,3,3,3
2,4,3,3,4,4,4,4,4,4,3,3,2,4,4,4,3,3,4,4,4
3,4,4,3,3,3,4,4,3,4,4,3,2,3,3,3,3,3,3,4,3
4,4,4,3,3,4,4,3,4,3,4,4,2,3,3,3,3,3,3,3,3


Change the names of the columns to make it easier to read

In [3]:
col_list = []

for i in range(1,len(df.columns) + 1):
    col_list.append(f"Question {i}")
    
df.columns = col_list

Make a new column "Total Score" which is the sum of all scores

In [4]:
df["Total Score"] = df[list(df.columns)].sum(axis=1)
df.head()

,Question 1,Question 2,Question 3,Question 4,Question 5,Question 6,Question 7,Question 8,Question 9,Question 10,...,Question 12,Question 13,Question 14,Question 15,Question 16,Question 17,Question 18,Question 19,Question 20,Total Score
0,3,3,3,4,4,4,3,3,3,4,...,2,3,3,3,3,3,3,3,3,64
1,4,4,3,3,4,3,3,3,3,4,...,3,4,3,4,3,3,3,3,3,67
2,4,3,3,4,4,4,4,4,4,3,...,2,4,4,4,3,3,4,4,4,72
3,4,4,3,3,3,4,4,3,4,4,...,2,3,3,3,3,3,3,4,3,66
4,4,4,3,3,4,4,3,4,3,4,...,2,3,3,3,3,3,3,3,3,66


# Validity Test

Make a function to automatically return r table value from the excel file based on the alpha and degree of freedom

In [5]:
r_table_list = []
alpha = 0.05
N = len(df)

def r_table(alpha,N):
    tabel_r = pd.read_excel("R Tabel.xlsx")
    tabel_r.columns = ["df","t (0.05)","r (0.05)","t (0.01)","r (0.01)"]
    tabel_r = tabel_r.iloc[7:].set_index("df")
    
    if alpha == 0.05:
        return tabel_r.loc[N]["r (0.05)"]
    elif alpha == 0.01:
        return tabel_r.loc[N]["r (0.01)"]
    
for i in range(0,len(df.columns) - 1):
    r_table_list.append(r_table(alpha,N))

Find the value of r

In [6]:
r = df.corr()["Total Score"].values[:-1]

Determine whether each question is valid or not

In [7]:
description = []

for i,j in zip(r,r_table_list):
    if abs(i) < j:
        description.append("Not Valid")
    else:
        description.append("Valid")

Make a table to show the result of the validity test

In [8]:
validity_test = pd.DataFrame({"Question":range(1,len(df.columns)),
                              "|r Value|":r,
                              "r Table":r_table_list,
                              "Description":description}).set_index("Question")
validity_test

,|r Value|,r Table,Description
Question,,,
1,0.585654,0.355046,Valid
2,0.763516,0.355046,Valid
3,0.750445,0.355046,Valid
4,0.772020,0.355046,Valid
5,0.763461,0.355046,Valid
6,0.733216,0.355046,Valid
7,0.720872,0.355046,Valid
8,0.861985,0.355046,Valid
9,0.783789,0.355046,Valid


# Reliability Test

In [9]:
cronbach_alpha = pg.cronbach_alpha(data=df.drop("Total Score",axis=1))[0]
print(f"The value for Cronbach’s Alpha is {cronbach_alpha}\n")

if cronbach_alpha >= 0.9:
    value = f"{cronbach_alpha} ≥ 0.9"
    consistency = "Excellent"
elif 0.8 <= cronbach_alpha < 0.9:
    value = f"0.8 ≤ {cronbach_alpha} < 0.9"
    consistency = "Good"
elif 0.7 <= cronbach_alpha < 0.8:
    value = f"0.7 ≤ {cronbach_alpha} < 0.8"
    consistency = "Acceptable"
elif 0.6 <= cronbach_alpha < 0.7:
    value = f"0.6 ≤ {cronbach_alpha} < 0.7"
    consistency = "Questionable"
elif 0.5 <= cronbach_alpha < 0.6:
    value = f"0.5 ≤ {cronbach_alpha} < 0.6"
    consistency = "Poor"
else:
    value = f"{cronbach_alpha} < 0.5"
    consistency = "Unacceptable"
    
print(f'Since we calculated Cronbach’s Alpha to be {value}, we would say that the internal consistency is "{consistency}"')

The value for Cronbach’s Alpha is 0.940561203113923

Since we calculated Cronbach’s Alpha to be 0.940561203113923 ≥ 0.9, we would say that the internal consistency is "Excellent"
